In [ ]:
from datetime import datetime, date, time
import pyotp
import datetime
from smartapi.smartConnect import SmartConnect
import time as tt
import requests
import pandas as pd
import login as l
pd.set_option('max_columns', None)


In [ ]:

######################################################
totp = pyotp.TOTP('')
totp = totp.now()
#####################################################


def intializeSymbolTokenMap():
    url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
    d = requests.get(url).json()
    global token_df
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'])
    token_df = token_df.astype({'strike': float})
    l.token_map = token_df


def getTokenInfo(exch_seg, instrumenttype, symbol, strike_price, pe_ce):
    df = l.token_map
    strike_price = strike_price*100

    if exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce)) & (df['expiry'] >= str(datetime.date.today()))].sort_values(by=['expiry'])
def place_order(token,symbol,qty,exch_seg,buy_sell,ordertype,price):
    try:
        orderparmas ={
            "variety": "NORMAL",
            "tradingsymbol": symbol,
            "symboltoken": token,
            "transactiontype": buy_sell,
            "exchange": exch_seg,
            "ordertype": ordertype,
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": price,
            "squareoff": "0",
            "stoploss": "0",
            "quantity": qty
            }
        orderId=obj.placeOrder(orderparmas)
        print("The order id is: {}".format(orderId))
    except Exception as e:
        print("Order placement failed: {}".format(e.message))


def getCandleData(symbolInfo):

    try:
        historicParam = {
            "exchange": symbolInfo.exch_seg,
            "symboltoken": symbolInfo.token,
            "interval": "ONE_MINUTE",
            "fromdate": f'{date.today()-datetime.timedelta(90)} 09:15',
            "todate": f'{date.today()} 15:30',
            # -datetime.timedelta(1)
        }
        res_json = obj.getCandleData(historicParam)
        columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        df = pd.DataFrame(res_json['data'], columns=columns)
        df['timestamp'] = pd.to_datetime(
            df['timestamp'], format='%Y-%m-%dT%H:%M:%S')
        df['symbol'] = symbolInfo.symbol
        df['expiry'] = symbolInfo.expiry
        print(f"Done for {symbolInfo.symbol}")
        # print(df.tail(1))
        # df.to_csv("Options_DATA_nifty_1min_CE.csv")
        return df
    except Exception as e:
        print(f"Historic Api failed: {e} {symbolInfo.symbol}")


obj = SmartConnect(api_key=l.api_key)
token = obj.generateToken(obj.generateSession(
    l.user_id, l.password, totp)["data"]["refreshToken"])
jwtToken = token['data']["jwtToken"]
refreshToken = token['data']['refreshToken']
feedToken = token['data']['feedToken']
# fetch the feedtoken
feedToken = obj.getfeedToken()
l.feed_token = feedToken

intializeSymbolTokenMap()

tokenInfo1=getTokenInfo('NFO', 'OPTIDX', 'NIFTY', 18400, 'CE').iloc[1]
tokenInfo2=getTokenInfo('NFO', 'OPTIDX', 'NIFTY', 18350, 'PE').iloc[1]
tokenInfo3=getTokenInfo('NFO', 'OPTIDX', 'NIFTY', 18300, 'CE').iloc[0]
tokenInfo4=getTokenInfo('NFO', 'OPTIDX', 'NIFTY', 18300, 'PE').iloc[0]
tokenInfo5=getTokenInfo('NFO', 'OPTIDX', 'NIFTY', 18350, 'CE').iloc[0]
tokenInfo6=getTokenInfo('NFO', 'OPTIDX', 'NIFTY', 18250, 'PE').iloc[0]
symbol1 = tokenInfo1['symbol']
token1 = tokenInfo1['token']
symbol2 = tokenInfo2['symbol']
token2 = tokenInfo2['token']
c=getCandleData(tokenInfo1)
b=getCandleData(tokenInfo2)
ltp = obj.ltpData('NFO', symbol1, token1)
place_order(token1,symbol1,50,'NFO','BUY','MARKET',0)
Ltp = ltp['data']['ltp']
print(Ltp)
ltp1 = obj.ltpData('NFO', symbol2, token2)
Ltp1 = ltp1['data']['ltp']
print(Ltp1)

In [ ]:
import time 
from datetime import  datetime
import datetime
timenow = datetime.datetime.now().minute
s_time = int(timenow)
e_time = int(59)
from IPython.display import clear_output
frange=[0,0]

while True:
    timenow = datetime.datetime.now().minute
    print("CURRENT TIME:", time.asctime(time.localtime(time.time())))
    if timenow == s_time:
      a = obj.ltpData('NFO', symbol1, token1)['data']['ltp']
      b = obj.ltpData('NFO', symbol2, token2)['data']['ltp']
      while timenow < e_time: 
            timenow = datetime.datetime.now().minute    
            print("CURRENT TIME:", time.asctime(time.localtime(time.time())))
            ltp = obj.ltpData('NFO', symbol1, token1)
            ltp1 = obj.ltpData('NFO', symbol2, token2)
            Ltp = ltp['data']['ltp']
            Ltp1 = ltp1['data']['ltp']
            x=0
            change1=((Ltp-a)/a)*100
            change2=((Ltp1-b)/b)*100
            if change1>=0 and change2>=0:
                if change1>change2:
                  x=change1+change2
                  if x>frange[0]:
                    frange[0]=x
                  print(x,frange[0],frange[1])
            if change1>=0 and change2<=0 or change1<=0 and change2>=0:
                  x=change1+change2 
                  if x>frange[0]:
                    frange[0]=x
                  if x<frange[1]:
                     frange[1]=x     
                  print(x,frange[0],frange[1])   
            if change1<=0 and change2<=0:
                if change1>change2:
                  x=change2+change1
                  if x<frange[1]:
                     frange[1]=x     
                  print(x,frange[0],frange[1])       
            print(change1,change2)              
            print(Ltp, Ltp1)
            print(a,b)
            time.sleep(1)
            clear_output(wait=True)
    clear_output(wait=True)
    time.sleep(1)